In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn

In [ ]:
train_loc = "../input/digit-recognizer/train.csv"
test_loc = "../input/digit-recognizer/test.csv"

In [ ]:
df_train = pd.read_csv(train_loc)
df_test = pd.read_csv(test_loc)

In [ ]:
df_train.head()

In [ ]:
X = df_train.drop(columns=['label'], axis=1).values/255
y = df_train['label'].values
X_test = df_test.values/255

In [ ]:
X_train_tensor = torch.from_numpy(X).type(torch.FloatTensor)
y_train_tensor = torch.from_numpy(y).type(torch.LongTensor)

X_test_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)

In [ ]:
train = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
test = torch.utils.data.TensorDataset(X_test_tensor)

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size = 100, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 100, shuffle = False)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = nn.Conv2d(1, 32, (5,5), 1)
        self.c2 = nn.Conv2d(32, 32, (5,5), 1)
        self.c3 = nn.Conv2d(32, 64, (3,3), 1)
        self.c4 = nn.Conv2d(64, 64, (3,3), 1)
        self.fc1 = nn.Linear(576, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d((2,2))
        self.max_pool2 = nn.MaxPool2d((2,2), (2,2))
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.relu(self.c1(x))
        x = self.relu(self.c2(x))
        x = self.dropout1(self.max_pool1(x))
        x = self.relu(self.c3(x))
        x = self.relu(self.c4(x))
        x = self.dropout1(self.max_pool2(x))
        x = self.dropout2(self.relu(self.fc1(x.view(x.size(0), -1))))
        x = self.fc2(x)
        
        return x

In [ ]:
model = Net()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    running_loss = 0
    
    for i, (img, labels) in enumerate(train_loader):
        img = img.view(-1,1,28,28)
        img = img.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch:{epoch+1}, loss:{running_loss/len(train_loader)}")

In [ ]:
acc_score = 0
for val_img, val_labels in train_loader:
    val_img = val_img.to(device)
    val_labels = val_labels.to(device)
    
    pred = model(val_img.view(-1,1,28,28))
    pred = pred.argmax(axis=1)
    
    acc = accuracy_score(pred.cpu(), val_labels.cpu())
    acc_score += acc
    
print(f"Accuracy in validation data is {acc_score/len(train_loader)}")

In [ ]:
torch.save(model.state_dict(),'mnist_model2.pt')

In [ ]:
model2 = Net()
model2.load_state_dict(torch.load('mnist_model.pt'))

In [ ]:
model2.to(device)
model2.eval()

In [ ]:
acc_score = 0
for val_img, val_labels in val_loader:
    val_img = val_img.to(device)
    val_labels = val_labels.to(device)
    
    pred = model2(val_img.view(-1,1,28,28))
    pred = pred.argmax(axis=1)
    
    acc = accuracy_score(pred.cpu(), val_labels.cpu())
    acc_score += acc
    
print(f"Accuracy in validation data is {acc_score/len(val_loader)}")